In [ ]:
from selenium import webdriver
import itertools
import pandas as pd
import re
import matplotlib.pyplot as plt
import mplcursors
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import flipkart

chrome_browser = webdriver.Chrome('C:/Users/Admin/Downloads/chromedriver_win32/chromedriver.exe')

print('Enter 1 if you want to search for Book,Enter 2 if you want to search for Electronics:: ', end='')
choice = int(input())

print('What do you want to search for ? : ', end='')
query_string = input()
url = 'https://www.amazon.in/s?k=' + query_string + '&page='

print('What keyword do you want apply for? : ')
print('If you want to apply more than one keyword then use | between words,like below example,')
print('Suppose I 3 keywords, "xy","ab","he llo" then,give input like this: xy|ab|he llo')
keyword = input()
flipkart_product_name, flipkart_price, flipkart_links = flipkart.check_for_flipkart(choice, query_string, keyword)

product_name = []
price_list = []
link_list = []
for i, j, k in zip(flipkart_product_name, flipkart_price, flipkart_links):
    print(f'{i} @ {j}\n')
    product_name.append(i)
    price_list.append(j)
    link_list.append(k)

j = 0
count = 0
for i in range(3):

    url1 = ""
    url1 = url + str(i)
    chrome_browser.get(url1)

    element = chrome_browser.find_elements_by_xpath("//span[@class='a-size-medium a-color-base a-text-normal']")  # product name
    # rating =chrome_browser.find_elements_by_xpath("//span[@class='a-size-base']")
    # gives total number of persons who have give their ratings

    for_accurate_price = chrome_browser.find_elements_by_css_selector('.s-no-outline')

    for m in for_accurate_price:
        link_list.append(m.get_attribute('href'))
        count += 1

    for ele in element:
        # print(f'{j} Page no.{i}. {ele.text}')
        product_name.append(ele.text)
        # price_list.append(pricee.text)
        # print()
        j += 1

print(f'len(link_list) = {len(link_list)},  len(product_name) = {len(product_name)}')

# for i in link_list:
#     print(i)

count = 0
file_product_name = []
file_link_list = []

if choice == 2:
    for (name, link) in zip(product_name, link_list):
        chrome_browser.get(link)
        try:
            element = chrome_browser.find_element_by_xpath("//span[@id='productTitle']")
            price = chrome_browser.find_element_by_xpath("//span[@id='priceblock_ourprice']")  # product name
            if re.search(keyword, element.text, re.IGNORECASE):
                file_product_name.append(element.text)
                price_list.append(price.text)
                file_link_list.append(link)
        except Exception:
            try:
                price = chrome_browser.find_element_by_xpath("//span[@id='priceblock_saleprice']")
                if re.search(keyword, element.text, re.IGNORECASE):
                    file_product_name.append(element.text)
                    price_list.append(price.text)
                    file_link_list.append(link)
            except Exception:
                try:
                    price = chrome_browser.find_element_by_xpath("//span[@id='priceblock_dealprice']")
                    if re.search(keyword, element.text, re.IGNORECASE):
                        file_product_name.append(element.text)
                        price_list.append(price.text)
                        file_link_list.append(link)
                except Exception:
                    count += 1
elif choice == 1:
    for (name, link) in zip(product_name, link_list):
        chrome_browser.get(link)
        try:
            element = chrome_browser.find_element_by_xpath("//span[@id='productTitle']")
            price = chrome_browser.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")  # product name
            if re.search(keyword, element.text, re.IGNORECASE):
                file_product_name.append(element.text)
                price_list.append(price.text)
                file_link_list.append(link)
        except Exception:
            count += 1

print(f'exception count = {count}')
print(f'len(file_product_name) = {len(file_product_name)}, len(price_list) = {len(price_list)}')

for (i, j) in zip(file_product_name, price_list):
    print('{} @{}\n'.format(i, j))


file = pd.DataFrame({'Product Name': file_product_name,
                    'Link': file_link_list,
                     'Price': price_list})

file['Price'] = file['Price'].str.replace('[\₹\,\.]','').astype(int)  #to remove Rupee sign from this Price column and to make it as int
file['Price'] = file['Price']/100

fig , ax = plt.subplots(figsize=(10,7))
ax.scatter(file['Product Name'],file['Price'])
crs = mplcursors.cursor(ax,hover=True)
crs.connect("add", lambda sel: sel.annotation.set_text('Point {},{}'.format(sel.target[0]+3, sel.target[1])))

# plt.xticks(file['Product Name'],"")  # this will remove x-axis labels from the plot
ax.axes.get_xaxis().set_visible(False)
plt.style.use('seaborn')

file.to_csv('Amazon.csv', index=False)

plt.show();

print('Provide a link,for which would you like to analyze comment :: ')
chosen_link = input()
boolean = True
comment_list = []
page_index = 1
if 'flipkart' in chosen_link:
    comment_list = flipkart.check_for_flipkart_comment(chosen_link)

else:

    while boolean:
        url = chosen_link + '&pageNumber=' + str(page_index)
        try:
            chrome_browser.get(url)
            comments = chrome_browser.find_elements_by_xpath("//span[@data-hook='review-body']")
            page_index += 1
            for i in comments:
                print(i.text)
                print()
                if i.text in comment_list:
                    break
                comment_list.append(i.text)
        except Exception:
            boolean = False
        if page_index > 75:
            boolean = False
file2 = pd.DataFrame()
file2['Comments'] = comment_list
file2.to_csv('Comment.csv', index=False)

sid = SentimentIntensityAnalyzer()

sentiment = []
sentiment_for_graph = {'pos':0,'neg':0,'neutral':0}
file = pd.read_csv('Comment.csv')
for i in file['Comments']:
    b = sid.polarity_scores(i)
    if b['compound']>=0.05:
        sentiment.append('pos')
        sentiment_for_graph['pos']+=1
    elif -0.05 < b['compound'] < 0.05:
        sentiment.append('neutral')
        sentiment_for_graph['neutral']+=1
    else:
        sentiment.append('negative')
        sentiment_for_graph['neg']+=1
file3 = pd.DataFrame({'Comments':comment_list,'Sentiment':sentiment})
file3.to_csv('Comment.csv',index=False)

total = sentiment_for_graph['pos'] + sentiment_for_graph['neg'] + sentiment_for_graph['neutral']
pos_percent = sentiment_for_graph['pos']*100/total
neg_percent = sentiment_for_graph['neg']*100/total
neutral_percent = sentiment_for_graph['neutral']*100/total

fig , (ax1,ax2) = plt.subplots(nrows=1, ncols=2,figsize =(10,5))
# %matplotlib nbagg
ax1.bar(sentiment_for_graph.keys(),sentiment_for_graph.values())
ax1.set(title='Count-wise Bar plot')
plt.style.use('seaborn')
crs = mplcursors.cursor(ax1,hover=True)
crs.connect("add", lambda sel: sel.annotation.set_text('Total Count: {}'.format(sel.target[1])))

# plt.style.use('ggplot')
ss = {'pos':pos_percent,'neg':neg_percent,'neutral':neutral_percent}
ax2.bar(ss.keys(),ss.values())
ax2.set(title='Percent-wise Bar plot')
crs = mplcursors.cursor(ax2,hover=True)
crs.connect("add", lambda sel: sel.annotation.set_text('Percentage: {}%'.format(sel.target[1])));